In [1]:
import tensorflow as tf

In [2]:
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()

Instructions for updating:
Use the retry module or similar alternatives.


In [3]:
import pandas as pd
import numpy as np
import time

In [4]:
dataset=pd.read_csv("cancer_detection.csv")

In [5]:
dataset.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [6]:
dataset=dataset.drop(['id','Unnamed: 32'],axis=1)

In [7]:
dataset.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
diagnosis                  569 non-null object
radius_mean                569 non-null float64
texture_mean               569 non-null float64
perimeter_mean             569 non-null float64
area_mean                  569 non-null float64
smoothness_mean            569 non-null float64
compactness_mean           569 non-null float64
concavity_mean             569 non-null float64
concave points_mean        569 non-null float64
symmetry_mean              569 non-null float64
fractal_dimension_mean     569 non-null float64
radius_se                  569 non-null float64
texture_se                 569 non-null float64
perimeter_se               569 non-null float64
area_se                    569 non-null float64
smoothness_se              569 non-null float64
compactness_se             569 non-null float64
concavity_se               569 non-null float64
concave points_se          569 n

In [9]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset['diagnosis']=encoder.fit_transform(dataset['diagnosis'])

In [10]:
dataset['diagnosis'].value_counts()

0    357
1    212
Name: diagnosis, dtype: int64

In [11]:
df=dataset.values

In [12]:
df[:,0:10]

array([[ 1.     , 17.99   , 10.38   , ...,  0.3001 ,  0.1471 ,  0.2419 ],
       [ 1.     , 20.57   , 17.77   , ...,  0.0869 ,  0.07017,  0.1812 ],
       [ 1.     , 19.69   , 21.25   , ...,  0.1974 ,  0.1279 ,  0.2069 ],
       ...,
       [ 1.     , 16.6    , 28.08   , ...,  0.09251,  0.05302,  0.159  ],
       [ 1.     , 20.6    , 29.33   , ...,  0.3514 ,  0.152  ,  0.2397 ],
       [ 0.     ,  7.76   , 24.54   , ...,  0.     ,  0.     ,  0.1587 ]])

In [13]:
# Read the data into a dataset.
dataset = tf.data.Dataset.from_tensor_slices((df[:,1:30], df[:,0]))

In [14]:
# Create variables.
w = tfe.Variable(tf.random_normal([30,1]))
b = tfe.Variable(0.0)

In [15]:
# Defining the model
def model(x):
    return x * w + b

In [16]:
def loss_function(y, y_predicted):
    return tf.nn.softmax_cross_entropy_with_logits(labels=y_predicted, logits=y)

In [17]:
def train(loss_fn):
    """Train a regression model evaluated using `loss_fn`."""
    print('Training; loss function: ' + loss_fn.__name__)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

In [18]:
# Define the function through which to differentiate.
def loss_for_example(x, y):
    return loss_function(y, model(x))

In [19]:
grad_fn = tfe.implicit_value_and_gradients(loss_for_example)

In [ ]:
start = time.time()
for epoch in range(100):
    #total_loss = 0.0
    for x_i, y_i in tfe.Iterator(dataset):
        loss, gradients = grad_fn(x_i, y_i)
        # Take an optimization step and update variables.
        optimizer.apply_gradients(gradients)
        #total_loss += loss
    if epoch % 10 == 0:
        print('Epoch {0}: {1}'.format(epoch,loss))
    print('Took: %f seconds' % (time.time() - start))
    print('Eager execution exhibits significant overhead per operation. '
        'As you increase your batch size, the impact of the overhead will '
        'become less noticeable. Eager execution is under active development: '
        'expect performance to increase substantially in the near future!')

train(Vloss_function)
#plt.plot(data[:,0], data[:,1], 'bo')
# The `.numpy()` method of a tensor retrieves the NumPy array backing it.
# In future versions of eager, you won't need to call `.numpy()` and will
# instead be able to, in most cases, pass Tensors wherever NumPy arrays are
# expected.
#plt.plot(data[:,0], data[:,0] * w.numpy() + b.numpy(), 'r',
        # label="huber regression")
#plt.legend()
#plt.show()